In [ ]:
!nvidia-smi

First, we grab matplotlib, and set the old "classic" style for some reason only Rui knows.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('classic')

In [ ]:
import numpy as np
import torch

These are the local imports. Make sure you import the correct model!

In [ ]:
from model.models import SimpleCNN2Layer as Model
from model.collectdata import collect_data
from model.training import select_gpu
from model.plots import plot_ruiplot
from model.efficiency import pv_locations, efficiency

Select a GPU here. Same numbering as the NVidia-SMI tool.

In [ ]:
device = select_gpu(0)

Pick a file to load.

In [ ]:
validation = collect_data('data/Oct03_20K_val.h5',
                          batch_size=1,
                          slice=slice(100),
                          masking=True,
                          device=device)

> Note: to get the real PV locations, use `collect_truth('file.h5', pvs=True)` to collect PVs (or SVs with `pvs=False`)

Let's just see how many NaNs we have in the dataset.

In [ ]:
print(*np.sum(np.isnan(validation.dataset.tensors[1].cpu().numpy()), 1))

Select a model to load. Make sure it matches the model you imported above.

In [ ]:
name = '../notebooks/Sep_18_mask_120000_2layer/Sep_18_mask_120000_2layer_5.pyt'
model = Model().to(device)
model.load_state_dict(torch.load(name))
model.eval()

Let's grab the outputs and labels as normal numpy arrays.

In [ ]:
%%time
with torch.no_grad():
    outputs = model(validation.dataset.tensors[0]).cpu().numpy()
    labels = validation.dataset.tensors[1].cpu().numpy()

And here's Rui's plotting code.

In [ ]:
inputs = validation.dataset.tensors[0].cpu().numpy().squeeze()
zvals = np.linspace(-100, 300, 4000, endpoint=False) + 0.05
finalmsg = ''
internal_count = 0

for n in range(10):
    input = inputs[n]
    label = labels[n]
    output = outputs[n]
    
    parameters = {
        "threshold": 1e-2,
        "integral_threshold": .2,
        "min_width": 3
    }
    
    ftruth = pv_locations(label, **parameters)
    fcomputed = pv_locations(output, **parameters)
    results = efficiency(label, output, difference=5.0, **parameters)
    
    finalmsg += str(results) + '\n'
    
    truth = np.around(ftruth).astype(np.int32)
    computed = np.around(fcomputed).astype(np.int32)
    
    # Join arrays and remove any points closer than 5 bins
    poi = np.sort(np.concatenate([truth, computed]))
    poi = poi[np.concatenate([[True], np.fabs(np.diff(poi)) > 5])]
    
    for i in poi:
        b_truth = np.fabs(ftruth - i) <= 5
        b_comp = np.fabs(fcomputed - i) <= 5
        in_truth = np.any(b_truth)
        in_comp = np.any(b_comp)
        
        if in_truth and in_comp:
            msg = 'PV found'
        elif in_truth:
            msg = 'PV not found'
        else:
            msg = 'False positive'
            
        with plt.style.context({
            'font.size':18,
            'font.weight':'bold'}):
        
            ax1, ax2 = plot_ruiplot(zvals, i, input, label, output)
            ax1.set_title(f"Event {n}: {msg}", fontdict={'size':18, 'weight':'bold'})

            v = .8

            truth_centroid = (ftruth[b_truth] / 10) - 100
            for value in truth_centroid:
                ax1.text(.02, v, f"True: {value:.3f} mm",
                         transform=ax1.transAxes)
                v -= .07

            comp_centroid = (fcomputed[b_comp] / 10) - 100
            for value in comp_centroid:
                ax1.text(.02, v, f"Pred: {value:.3f} mm",
                         transform=ax1.transAxes)
                v -= .07
                
            if len(truth_centroid) == 1 and len(comp_centroid) == 1:
                diff = np.fabs(truth_centroid[0] - comp_centroid[0]) * 1_000
                ax1.text(.02, v, f"∆: {diff:.0f} µm",
                         transform=ax1.transAxes)
                v -= .07

            plt.savefig(f'120000_3layer_{internal_count:02}.pdf')
            plt.show()
            internal_count += 1
            
print(finalmsg)

In [ ]:
exit()